# Processamento de Linguagem Natural

In [1]:
# import nltk

# nltk.download()  
"""
    *** ATENÇÃO ***

   Descomente as linhas acima se for usar NLTK pela primeira vez
"""

'\n    *** ATENÇÃO ***\n\n   Descomente as linhas acima se for usar NLTK pela primeira vez\n'

## Carregando os dados

In [1]:
import pandas as pd

df_pos = pd.read_csv("reviews_curtos/positive.txt", delimiter="\n", encoding="latin", header=None, names=['text'])
df_pos['positive'] = 1
df_pos.head()

,text,positive
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [2]:
df_neg = pd.read_csv("reviews_curtos/negative.txt", delimiter="\n", encoding="latin", header=None, names=['text'])
df_neg['positive'] = 0
df_neg.head()

,text,positive
0,"simplistic , silly and tedious .",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0


## Concatenando os dois dataframes

In [3]:
from sklearn.utils import shuffle
df = shuffle(pd.concat([df_pos, df_neg]))
df.head()

,text,positive
914,"from spiritual rebirth to bruising defeat , vi...",1
3438,seeks to transcend its genre with a curiously ...,1
927,mindless and boring martial arts and gunplay w...,0
3328,control-alt-delete simone as quickly as possible,0
2882,"the movie is as far as you can get from racy ,...",0


In [4]:
df['positive'].value_counts()

1    5331
0    5331
Name: positive, dtype: int64

## Funções auxiliares (tokenização, stopwords, steaming, classes de palavras...)

In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import ne_chunk
from nltk.corpus import stopwords
from string import punctuation
# import spacy
from nltk.tag import pos_tag
import re

def tokenizacao(texto):
    #Tokenização
    tokenizado = word_tokenize(texto.lower())
    return tokenizado

def remove_stopwords(texto_tokenizado):
    #Remoção das stopwords e pontuações
    stopwords_ = set(stopwords.words('english') + list(punctuation))
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado 
                              if palavra not in stopwords_]
    return palavras_sem_stopwords
        
def faz_steaming(texto_tokenizado):
    stemmer = PorterStemmer()
    palavras_steaming = [stemmer.stem(palavra) 
                         for palavra in texto_tokenizado]
    return palavras_steaming

def chunk_chik(texto_tokenizado, chunkGram):
    #Chunk e Chiking
    palavras_post = pos_tag(texto_tokenizado)
    palavras_chunk = [palavra for palavra, classe in palavras_post 
                      if not re.match(chunkGram, palavra)]
    return palavras_chunk
    
def pipeline_npl(texto):
    tokenizado = tokenizacao(texto)
    palavras_sem_stopwords = remove_stopwords(tokenizado)
    palavras_post = faz_steaming(palavras_sem_stopwords)
    
    palavras_chunk = chunk_chik(
        palavras_post,
        r"""Chunk: {<VB.?>*<VB.?>*<NNP>+<NN>?}"""
    )

    return " ".join(palavras_chunk)

## Testando primeiro o *pipeline* em apenas uma frase

In [6]:
pipeline_npl('I am Batman. Get out of Gotham City!!')

'batman get gotham citi'

## Aplicando o processamento em todos os dados da coluna *text*
### `-- Demora um pouco --`

In [7]:
df['text'] = df['text'].apply(pipeline_npl)
df.head()

,text,positive
914,spiritu rebirth bruis defeat vincent 's odysse...,1
3438,seek transcend genr curious styliz quasi-shake...,1
927,mindless bore martial art gunplay littl excit ...,0
3328,control-alt-delet simon quickli possibl,0
2882,movi far get raci point almost stop blood flow...,0


# Criando os modelos de classificação

In [9]:
from sklearn import metrics

#Teste de precisão
def avalia_modelo(modelo):
    modelo.fit(x_treino_dtm, y_treino)
    y_predito = modelo.predict(x_teste_dtm)
    precisao = metrics.accuracy_score(y_teste, y_predito)*100
    print("Precisão do modelo {}: {:.2f}%".format(
        modelo.__class__.__name__,
        precisao
    ))
    return modelo

In [10]:
from sklearn.model_selection import train_test_split
#Separação das entradas e saidas
x = df["text"]
y = df["positive"].values
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                        test_size=0.4,
                                                        random_state=15)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

#Método de Vetorização
vetor = CountVectorizer()
vetor.fit(x_treino)

x_treino_dtm = vetor.transform(x_treino.values)
x_teste_dtm = vetor.transform(x_teste.values)

In [12]:
from sklearn.naive_bayes import MultinomialNB

## Classificando a emoção (positiva / negativa):

In [13]:
frase_proc = processamento_npl(
    '''
    Seems not good.
    '''
)
x_novo = vetor.transform([frase_proc])
nb_clf = avalia_modelo(MultinomialNB())
pred = nb_clf.predict(x_novo)
print('-'*20, 'PREVISÃO', '-'*20)
tipos = {
    0: 'negative',
    1: 'positive'
}
tipos[pred[0]]


NameError: name 'processamento_npl' is not defined